In [1]:
W = trues(10000, 10000)
W[rand(1:length(W), round(Int, 0.1length(W)))] .= 0
W

10000×10000 BitMatrix:
 1  1  0  1  1  1  1  1  1  1  0  1  1  …  1  1  1  1  1  1  0  1  1  0  0  0
 1  1  1  1  1  1  1  1  1  1  1  0  1     0  0  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  0  1  1  1  1  1  0     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  0  0  1  1  1  1  1  1
 1  0  1  0  1  0  1  1  0  1  1  0  0     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  0  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  0  1  1  1  1  1  0  1
 1  1  1  1  1  1  1  1  1  1  1  1  0     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  0  1  0  1  1     1  1  1  1  1  1  1  1  1  0  1  1
 1  1  1  1  1  1  1  0  1  1  1  1  1     1  1  1  1  1  1  1  1  0  1  1  1
 1  1  0  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  0  1  1  1  1  0  1
 1  1  1  1  1  1  0  1  0  1  1  1  1     1  1  1  0  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  0  1  1  1  0  1  1   

In [2]:
function get_S(W)
    zeroidxs = getindex.(findall(iszero, W), [1 2])
    S1 = Set(zeroidxs[:,1])
    S2 = Set(zeroidxs[:,2])
    return S1, S2
end

get_S (generic function with 1 method)

In [3]:
@time get_S(W)
@time get_S(W)
@time get_S(W)

  1.122026 seconds (838.01 k allocations: 884.300 MiB, 15.69% gc time, 18.37% compilation time)
  0.747305 seconds (47 allocations: 835.382 MiB, 2.45% gc time)
  0.798846 seconds (47 allocations: 835.382 MiB, 9.23% gc time)


(Set([664, 7513, 5713, 1328, 4211, 4407, 2656, 3431, 8368, 8814  …  786, 790, 9551, 4888, 395, 8856, 9108, 4675, 8612, 611]), Set([664, 7513, 5713, 1328, 4211, 4407, 2656, 3431, 8368, 8814  …  786, 790, 9551, 4888, 395, 8856, 9108, 4675, 8612, 611]))

In [4]:
function get_S1(W)
    zeroidxs = getindex.(findall(iszero, W), [1 2])
    S1 = unique(zeroidxs[:,1])
    S2 = unique(zeroidxs[:,2])
    return S1, S2
end

get_S1 (generic function with 1 method)

In [5]:
@time get_S1(W)
@time get_S1(W)
@time get_S1(W)

  0.736688 seconds (112.07 k allocations: 555.212 MiB, 2.24% gc time, 6.55% compilation time)
  0.733525 seconds (105 allocations: 548.261 MiB, 8.66% gc time)
  0.696745 seconds (105 allocations: 548.261 MiB, 3.41% gc time)


([14, 24, 29, 48, 57, 86, 100, 113, 114, 130  …  2711, 814, 3704, 189, 1995, 3532, 6294, 5695, 2350, 2495], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 10000])

In [6]:
sort.(unique.(collect.(get_S(W)))) == sort.(get_S1(W))

true

In [7]:
function get_S2(W)
    m, n = size(W)
    b1, b2 = fill(false, m), fill(false, n)
    Threads.@threads for j in 1:n
        for i in 1:n
            @inbounds if W[i, j] == 0
                b1[i] = b2[j] = true
            end
        end
    end
    b1, b2
end

get_S2 (generic function with 1 method)

In [8]:
@time get_S2(W)
@time get_S2(W)
@time get_S2(W)

  0.073457 seconds (128.54 k allocations: 7.628 MiB, 46.67% compilation time)
  0.040549 seconds (184 allocations: 35.562 KiB)
  0.015305 seconds (65 allocations: 26.141 KiB)


(Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [9]:
get_S3(W) = findall.(get_S2(W))

get_S3 (generic function with 1 method)

In [10]:
@time get_S3(W)
@time get_S3(W)
@time get_S3(W)

  0.038580 seconds (84.05 k allocations: 5.480 MiB, 53.22% compilation time)
  0.015409 seconds (78 allocations: 182.938 KiB)
  0.015596 seconds (77 allocations: 182.766 KiB)


([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 10000], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 10000])

In [11]:
sort.(get_S1(W)) == get_S3(W)

true

In [12]:
get_S(W) |> typeof

Tuple{Set{Int64}, Set{Int64}}

In [13]:
get_S1(W) |> typeof

Tuple{Vector{Int64}, Vector{Int64}}

In [14]:
get_S2(W) |> typeof

Tuple{Vector{Bool}, Vector{Bool}}

In [15]:
get_S3(W) |> typeof

Tuple{Vector{Int64}, Vector{Int64}}